## Topics
1. `WHERE` clause
2. [Comparisons](https://duckdb.org/docs/sql/expressions/comparison_operators)
   1. Logical
   2. `IS NULL`
   3. `BETWEEN`
3. Filtering with `JOINS`
4. `LIKE` & `ILIKE` [comparisons](https://duckdb.org/docs/sql/functions/patternmatching)
   1. Mention GLOB, REGEX
5. `IN` https://duckdb.org/docs/sql/expressions/in.html
6. `ORDER BY`
7. `GROUP BY`

In [48]:
import duckdb

%load_ext sql
%config SqlMagic.displaylimit = 100

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
conn = duckdb.connect()
%sql conn --alias duckdb

In [3]:
%%sql
IMPORT DATABASE '../../data/nps';

Running query in 'duckdb'

Count
448


Creating fields you commonly filter on can drastically improve readability. String/column manipulation is great for that.

In [4]:
%%sql 
WITH thursday AS (
    SELECT
        p.name,
        closed_thurs.category,
        closed_thurs.thursday,
        COALESCE(closed_thurs.thursday, 'Open') as closed_open,
        NOT closed_thurs.thursday IS NULL as is_closed
    FROM nps_public_data.parks p
    LEFT JOIN nps_public_data.park_hours closed_thurs
        ON closed_thurs.park_id = p.id
        AND closed_thurs.thursday = 'Closed'
    WHERE 1 = 1
)
SELECT
    *
FROM thursday
WHERE is_closed
LIMIT 5

Running query in 'duckdb'

name,category,thursday,closed_open,is_closed
Eleanor Roosevelt,Val-Kill Cottage Tours,Closed,Closed,True
Morristown,Winter Hours,Closed,Closed,True
Freedom Riders,Anniston Greyhound Bus Depot,Closed,Closed,True
Black Canyon Of The Gunnison,East Portal,Closed,Closed,True
Golden Gate,Fort Point National Historic Site,Closed,Closed,True


We commonly use the `WHERE` clause to filter aggregate queries, but we can also do so in `JOIN`s. 

However, we need to be very careful with how joins work.

In [7]:
%%sql
SELECT
    p.name,
    vc.name as visitor_center_name
FROM nps_public_data.parks p
LEFT JOIN nps_public_data.visitorcenters vc
    ON p.parkcode = vc.parkcode
WHERE 1 = 1
-- Filter base query (parks) for national monument
    AND p.designation = 'National Monument'
-- Filter JOIN (!) for passport stamp locations.
-- what will happen to parks without visitor centers?
    AND vc.ispassportstamplocation
LIMIT 1

Running query in 'duckdb'

name,visitor_center_name
Statue Of Liberty,Liberty Island Information Center


How many rows are returned with/without the `LEFT JOIN`? What does that say about the number of parks we're querying? Why do you think that is?

In [9]:
%%sql
WITH filter_in_join AS (
    SELECT
        p.name,
        vc.name as visitor_center_name
    FROM nps_public_data.parks p
    INNER JOIN nps_public_data.visitorcenters vc
        ON p.parkcode = vc.parkcode
), filter_in_where AS (
    SELECT
        p.name,
        vc.name as visitor_center_name
    FROM nps_public_data.parks p
    LEFT JOIN nps_public_data.visitorcenters vc
        ON p.parkcode = vc.parkcode
    WHERE vc.parkcode IS NOT NULL
)
SELECT
    COUNT(*) as ct
FROM filter_in_join

UNION ALL

SELECT
    COUNT(*) as ct
FROM filter_in_where


Running query in 'duckdb'

ct
2820
2820


Some common ways of filtering data

1. Comparisons (`>`, `<`, `=`)
2. `BETWEEN`
3. `IN`
4. `IS NULL`
5. `LIKE` & `ILIKE` // `REGEXP`

Comparisons and `BETWEEN` are good for integers, but also timestamps and dates (as we'll see). `IN` can be helpful for lists of data, while `IS NULL` can help us when `NULL` values are a possibility.

`ILIKE`, `LIKE`, and `REGEXP` are all useful when pattern matching is at play.

We can filter numbers and dates with comparisons or between statements

In [36]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND recurrencedatestart > '2024-01-01'
    AND recurrencedatestart < '2024-01-23'
ORDER BY RANDOM()
LIMIT 2


Running query in 'duckdb'

title,parkfullname,category,isfree,description
Acadian Cultural Center - Louisiana Talks & Tales,Jean Lafitte National Historical Park and Preserve,Regular Event,True,"Join a ranger to learn about the history, culture, or environment of south Louisiana."
Acadian Cultural Center - Louisiana Talks & Tales,Jean Lafitte National Historical Park and Preserve,Regular Event,True,"Join a ranger to learn about the history, culture, or environment of south Louisiana."


In [35]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND recurrencedatestart BETWEEN '2024-01-01' AND '2024-01-23'
ORDER BY RANDOM()
LIMIT 2

Running query in 'duckdb'

title,parkfullname,category,isfree,description
Acadian Cultural Center - Youth Art Showcase,Jean Lafitte National Historical Park and Preserve,Regular Event,True,"Join us for this special exhibition of artwork from local students who placed in this year's Youth Art Showcase. The 2023 theme was ""Waterways of the Atchafalaya Area."""
Acadian Cultural Center - Louisiana Talks & Tales,Jean Lafitte National Historical Park and Preserve,Regular Event,True,"Join a ranger to learn about the history, culture, or environment of south Louisiana."


But we have to note, `BETWEEN` is _inclusive_~

In [34]:
%%sql
SELECT
    'between' as f,
    COUNT(*) as ct
FROM nps_public_data.events e
WHERE 1 = 1
    AND recurrencedatestart BETWEEN '2024-01-01' AND '2024-01-23'
GROUP BY f

UNION ALL

SELECT
    'greater than' as f,
    COUNT(*) as ct
FROM nps_public_data.events e
WHERE 1 = 1
    AND recurrencedatestart > '2024-01-01'
    AND recurrencedatestart < '2024-01-23'
GROUP BY f

Running query in 'duckdb'

f,ct
between,6
greater than,2


Another handy way to filter datasets is through string matching— if you're familiar with Python, you probably know regex, but SQL has a few other, simpler ways. First, `LIKE`:

In [42]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND title LIKE '%Stroll%'
LIMIT 5

Running query in 'duckdb'

title,parkfullname,category,isfree,description
Afternoon Stroll,Joshua Tree National Park,Regular Event,True,"Join a ranger for a 0.4 mile (0.6 km) guided walk! Learn about various topics like plants, animals, geology, cultural history, and more. Topics vary by ranger. Bring water, closed–toe shoes, and layers for this 45–minute walk.Difficulty: easy, unpaved trail"
Afternoon Stroll,Joshua Tree National Park,Regular Event,True,"Join a ranger for a 0.4 mile (0.6 km) guided walk! Learn about various topics like plants, animals, geology, cultural history, and more. Topics vary by ranger. Bring water, closed–toe shoes, and layers for this 45–minute walk.Difficulty: easy, unpaved trail"


But `LIKE` is case sensitive, so it's easy to miss results.

In [52]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND title LIKE '%hike%'
LIMIT 5

Running query in 'duckdb'

title,parkfullname,category,isfree,description


Instead, we can use `ILIKE`:

In [53]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND title ILIKE '%hike%'
LIMIT 5

Running query in 'duckdb'

title,parkfullname,category,isfree,description
A Hike Through The (Cactus) Forest (East District),Saguaro National Park,Regular Event,True,"Let your interest reach new heights and join us for a hike through the heart of the cactus forest, getting to know the giant cactus.Hike will up to 2 miles round trip on a flat trail. Bring sun protection and water. Hiking boots are recommended."
A Hike Through The (Cactus) Forest (East District),Saguaro National Park,Regular Event,True,"Let your interest reach new heights and join us for a hike through the heart of the cactus forest, getting to know the giant cactus.Hike will up to 2 miles round trip on a flat trail. Bring sun protection and water. Hiking boots are recommended."


Depending on your flavor of SQL, there might be other ways to pattern match. DuckDB also has `glob` matching & `regex` matching, too.

Read more [here](https://duckdb.org/docs/sql/functions/patternmatching.html).

Sometimes, we might need to construct a list to perform a more robust filter.

In [ ]:
%%sql
WITH park_states AS (
    SELECT 
        name, 
        states AS states_string, 
        split(states, ',') ::string[] AS states_list
    FROM nps_public_data.parks p
    )
SELECT 
    * 
FROM park_states
WHERE list_contains(states_list, 'UT')